In [1]:
import pandas as pd
import numpy as np

import re
import torch
import faiss
from bleurt import score

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM

In [2]:
df_articles = pd.read_csv('./df_data.csv')
df_articles

,text
0,\nКапитан сборной Аргентины Лионель Месси сооб...
1,\nЛидер сборной Аргентины Лионель Месси вряд л...
2,\nINTERFAX.RU - Аргентинский футболист Лионель...
3,\nЛионель Месси не планирует участвовать в чем...
4,\nКапитан сборной Аргентины по футболу Лионель...
...,...
95,\nТинькофф Банк объявил о запуске Центрального...
96,\n«Тинькофф» запускает частный университет со ...
97,\n14 июня 2023 года «Тинькофф» объявил о запус...
98,\nТинькофф банк запустит частный университет -...


In [3]:
articles_list = df_articles['text'].tolist()
articles_list

['\nКапитан сборной Аргентины Лионель Месси сообщил, что не будет участвовать в следующем чемпионате мира, который пройдет в Мексике, США и Канаде в 2026 году.\n\n«Это был мой последний чемпионат мира. Посмотрю, как пойдут дела, но в принципе нет, на следующий чемпионат мира я не поеду», — заявил Месси изданию Titan.\n\nВ декабре 2022 года 35-летний футболист стал победителем чемпионата мира 2022 года. Это был его пятый турнир в карьере.\n\nМесси выступает в сборной Аргентине с 2005 года. Всего за команду он провел 174 матча и забил 102 гола. Вместе со сборной он завоевал 8 трофеев.\n\nЭтим летом после двух сезонов в «ПСЖ» Месси станет игроком клуба МЛС «Интер Майами».\n',
 '\nЛидер сборной Аргентины Лионель Месси вряд ли сыграет на чемпионате мира 2026 года. Об этом 35-летний футболист рассказал в интервью китайской газете Titan Sports, сообщает Marca.\n\n«Думаю, что нет. Это был мой последний чемпионат мира. Посмотрю, как пойдут дела, но, в принципе, нет, на следующий чемпионат мира 

In [4]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum", use_fast=False)

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

In [6]:
def generate_text(articles):
    WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

    input_ids = tokenizer(
        [WHITESPACE_HANDLER(el) for el in articles],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=1024
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        min_length=700,
        no_repeat_ngram_size=3,
        num_beams=4,
        max_new_tokens=800
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return summary

In [7]:
new_article = generate_text(articles_list[:10])
new_article

'Лионель Месси объявил, что не будет участвовать в следующем чемпионате мира по футболу, который пройдет в Мексике, США и Канаде в 2026 году. Об этом говорится в сообщении, опубликованном на официальном сайте аргентинского футбольного клуба ПСЖ в пятницу, 27 апреля. Это означает, что Месси будет играть в Бразилии в последний чемпионат мира в этом году, когда он будет работать. Для Месси.  Таким была его последняя история. После того, как Месси провел последний турнир. Журналист и футболист уже давно знал, что его фанаты уже поговорили о том, как ему это было именно так. Вот и сам Месси в беседе с Би-би-си. Что делать с Месси, а также о своем последнем матче в футболе, он не поедет на следующий чемпионат, где он не участвует в турнире, в котором он был победителем чемпионата мира в декабре 2022 года, и что будет с ним еще в прошлом году - его последний сезон в карьере и почему его не будут принимать матчи с бразильским клубом ФИФА, Месси отказался от участия в чемпионате, которые пройду

In [22]:
def bleurt_score(references, candidates):
    checkpoint = "./bleurt/bleurt/test_checkpoint"    
    scorer = score.BleurtScorer(checkpoint)
    scr = scorer.score(references=references, candidates=candidates)
    
    return scr

In [27]:
def get_bleurt_scores(articles_list, new_article, start, fin):
    scores = []
    for i in range(start, fin):
        scores.append(bleurt_score([articles_list[i]], [new_article])[-1])
    return scores

In [34]:
scores = get_bleurt_scores(articles_list, new_article, start=0, fin=10)

INFO:tensorflow:Reading checkpoint ./bleurt/bleurt/test_checkpoint.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint dbleurt_tiny
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:dbleurt_tiny
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
INFO:tensorflow:Reading checkpoint ./bleurt/bleurt/test_checkpoint.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint dbleurt_tiny
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:dbleurt_tiny
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tenso

In [37]:
scores

[0.20630379021167755,
 0.1077081710100174,
 0.028205692768096924,
 0.16378729045391083,
 0.1492864042520523,
 0.06487347185611725,
 0.23243819177150726,
 0.10696132481098175,
 0.20630379021167755,
 0.20858293771743774]